In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
! unzip '/content/drive/MyDrive/updated_COCO_Dataset (1).zip' -d '/content'

Archive:  /content/drive/MyDrive/updated_COCO_Dataset (1).zip
replace /content/content/dataset_1/COCO_Dataset/train/annotations/instances_default.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
! pip install transformers
!python -m pip install pyyaml==5.1
import sys, os, distutils.core
# Note: This is a faster way to install detectron2 in Colab, but it does not include all functionalities (e.g. compiled operators).
# See https://detectron2.readthedocs.io/tutorials/install.html for full installation instructions
!git clone 'https://github.com/facebookresearch/detectron2'
dist = distutils.core.run_setup("./detectron2/setup.py")
!python -m pip install {' '.join([f"'{x}'" for x in dist.install_requires])}
sys.path.insert(0, os.path.abspath('./detectron2'))
! pip install supervision
! pip install openai==1.0.0
! pip install instructor

  Using cached PyYAML-5.1.tar.gz (274 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
fatal: destination path 'detectron2' already exists and is not an empty directory.
  Using cached openai-1.0.0-py3-none-any.whl (154 kB)
  Attempting uninstall: openai
    Found existing installation: openai 1.28.1
    Uninstalling openai-1.28.1:
      Successfully uninstalled openai-1.28.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
instru

In [ ]:
# Importing all the libraries

import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

import warnings

warnings.filterwarnings("ignore")
# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
from detectron2 import model_zoo
from detectron2.data.datasets import register_coco_instances
from detectron2.utils.visualizer import ColorMode

import shutil

from typing import Tuple, List
import supervision as sv
import torch

In [ ]:
# fn to annotate the bounding boxes on the image
def annotate1(image_source: np.ndarray, boxes: np.ndarray,  phrases: List[str],logits: List = None) -> np.ndarray:
    h, w, _ = image_source.shape

    # boxes = boxes * torch.Tensor([w, h, w, h])

    # xyxy = box_convert(boxes=boxes, in_fmt="cxcywh", out_fmt="xyxy").numpy()

    detections = sv.Detections(boxes)
    # print(phrases)

    if logits == None:
      labels = phrases
    else:

        labels = [
            f"{phrase} {logit:.2f}"
            for phrase, logit
            in zip(phrases, logits)
        ]

    box_annotator = sv.BoxAnnotator()
    annotated_frame = cv2.cvtColor(image_source, cv2.COLOR_RGB2BGR)
    annotated_frame = box_annotator.annotate(scene=annotated_frame, detections=detections, labels=labels)
    return annotated_frame

In [ ]:
# device = "cuda" if torch.cuda.is_available() else "cpu"
# providing dataset and registering it
# registering the data set
train_json_path = '/content/content/dataset_1/COCO_Dataset/train/annotationsinstances_default.json'
valid_json_path = '/content/content/dataset_1/COCO_Dataset/valid/annotationsinstances_default.json'

register_coco_instances("my_dataset_train", {}, train_json_path, "/content/content/dataset_1/COCO_Dataset/train/images")
register_coco_instances("my_dataset_val", {}, valid_json_path, "/content/content/dataset_1/COCO_Dataset/train/images")




AssertionError: Dataset 'my_dataset_train' is already registered!

In [ ]:
# creating the metadata for the dataset
MetadataCatalog.get("my_dataset_train").set(thing_classes=["Rust"])
MetadataCatalog.get("my_dataset_val").set(thing_classes=["Rust"])
metadata = MetadataCatalog.get("my_dataset_train")
metadata_val = MetadataCatalog.get("my_dataset_val")
print(metadata)
print(metadata_val)

# creating and setting the configuration parameter for model
cfg = get_cfg()
# cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_C4_3x.yaml"))
# cfg.DATASETS.TRAIN = ("my_dataset_train",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
# cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_C4_3x.yaml")
cfg.SOLVER.IMS_PER_BATCH = 2  # Increase this number based on your GPU memory
cfg.SOLVER.BASE_LR = 0.00025
cfg.SOLVER.MAX_ITER = 500
cfg.SOLVER.STEPS = []  # No LR steps, you can add steps if you want LR decay
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 16
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1


# model loading
cfg.MODEL.WEIGHTS = '/content/drive/MyDrive/rust/model_final.pth'  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.60   # set a custom testing threshold
predictor = DefaultPredictor(cfg)

Metadata(name='my_dataset_train', json_file='/content/content/dataset_1/COCO_Dataset/train/annotationsinstances_default.json', image_root='/content/content/dataset_1/COCO_Dataset/train/images', evaluator_type='coco', thing_classes=['Rust'])
Metadata(name='my_dataset_val', json_file='/content/content/dataset_1/COCO_Dataset/valid/annotationsinstances_default.json', image_root='/content/content/dataset_1/COCO_Dataset/train/images', evaluator_type='coco', thing_classes=['Rust'])
[05/13 09:16:19 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from /content/drive/MyDrive/rust/model_final.pth ...


For Identifying the rust using trained model

In [1]:
# Rust Identfication to identify the rust , using our model here
def rust_identification(im):
      '''
      Return:
      bounding boxes
      im :size 1024 x 1024
      '''
      try:
        shutil.rmtree('/content/images')
        os.mkdir('/content/images')
      except:
          os.mkdir('/content/images')

      # im = cv2.imread('/content/drive/MyDrive/rust/Gemini_Generated_Image_7x4h7d7x4h7d7x4h.jpg')
      # im = cv2.resize(im, (1024, 1024))
      # print(im)
      outputs = predictor(im)  # format is documented at https://detectron2.readthedocs.io/tutorials/models.html#model-output-format
      v = Visualizer(im[:, :, ::-1],
                      metadata=metadata,
                      scale=0.5,
                      instance_mode=ColorMode.IMAGE   # remove the colors of unsegmented pixels. This option is only available for segmentation models
      )
      out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
      # cv2_imshow(out.get_image()[:, :, ::-1])


      # Assuming 'instances' contains the Instances object returned by model inference
      # Move the tensors from GPU to CPU
      instances = outputs["instances"]
      pred_boxes = instances.pred_boxes.tensor.cpu().numpy()
      scores = instances.scores.cpu().numpy()
      pred_classes = instances.pred_classes.cpu().numpy()
      bounding_boxes = []
      for i, bbox in enumerate(pred_boxes):
          x1, y1, x2, y2 = bbox
          bounding_boxes.append([x1, y1, x2, y2])
      print(bounding_boxes)
      return bounding_boxes, im





#####################For minimizing the overlapping bounding boxes###########################################################


# Function to calculate the area of intersection between two rectangles
def intersection_area(rect1, rect2):
    x_overlap = max(0, min(rect1[2], rect2[2]) - max(rect1[0], rect2[0]))
    y_overlap = max(0, min(rect1[3], rect2[3]) - max(rect1[1], rect2[1]))
    return x_overlap * y_overlap

# Function to fill overlapping areas with black color
def fill_overlapping_areas(image, rectangles):
    mask = np.zeros_like(image)
    for i, rect1 in enumerate(rectangles):
        for j, rect2 in enumerate(rectangles):
            if i != j:
                area = intersection_area(rect1, rect2)
                if area > 0:
                    overlap_x1 = max(rect1[0], rect2[0])
                    overlap_y1 = max(rect1[1], rect2[1])
                    overlap_x2 = min(rect1[2], rect2[2])
                    overlap_y2 = min(rect1[3], rect2[3])
                    mask[int(overlap_y1):int(overlap_y2), int(overlap_x1):int(overlap_x2)] = 255
    image[mask == 255] = 0
    return image

# Function to calculate the percentage of black color within a bounding box
def black_area_percentage(image, rectangle):
    total_pixels = (rectangle[2] - rectangle[0]) * (rectangle[3] - rectangle[1])
    black_pixels = np.count_nonzero(image[int(rectangle[1]):int(rectangle[3]), int(rectangle[0]):int(rectangle[2])] == 0)
    return (black_pixels / total_pixels) * 100


# Based on the threshold value i.e. the percentage of overlapping area, we are removing the bounding box
def remove_box(bounding_boxes, image, threshold):
    max_black_area = -1
    index_to_remove = -1
    for i, box in enumerate(bounding_boxes):
        percentage_black = black_area_percentage(image, box)
        if percentage_black > threshold:
            area = (box[2] - box[0]) * (box[3] - box[1])
            if area > max_black_area:
                max_black_area = area
                index_to_remove = i
    if index_to_remove != -1:
        del bounding_boxes[index_to_remove]
    return bounding_boxes



#bounding_boxes = pred_boxes
# Iteratively remove bounding boxes with black color above 90 percent and highest area of black color
def preprocess_bbox(bounding_boxes):
  iteration = 1
  max_iterations = 10

  # Iteratively remove bounding boxes with black color above 90 percent and highest area of black color
  iteration = 1
  max_iterations = 10

  while iteration <= max_iterations:

      image_after_filling = np.ones((1024, 1024), dtype=np.uint8) * 255  # White background
      image_after_filling = fill_overlapping_areas(image_after_filling, bounding_boxes)

      # Calculate and display the percentage of black color for each bounding box
      for i, box in enumerate(bounding_boxes):
          percentage_black = black_area_percentage(image_after_filling, box)

      # bounding_boxes = remove_box(bounding_boxes, image_after_filling)
      bounding_boxes = remove_box(bounding_boxes, image_after_filling, threshold=90)

      # Break the loop if there's only one bounding box left
      if len(bounding_boxes) == 1:
          break

      iteration += 1
  return bounding_boxes


# Crop the bounding boxes from the original image
def crop_bounding_box(image, bounding_box):
    # Convert bounding box coordinates to integers

    x1, y1, x2, y2 = map(int, bounding_box)

    # Crop the region of interest (ROI) from the original image
    cropped_image = image[y1:y2, x1:x2]

    return cropped_image

# Example usage:

# Read the original image and cropping and save the image based on the bounding boxes
# work as the main function for this block
def saving_croppedImg(im):
    # getting the bounding boxes and read image by using the model
    bounding_boxes, original_image = rust_identification(im)
    print(len(bounding_boxes))
    #Remove the overlapping bounding boxes
    bounding_boxes = preprocess_bbox(bounding_boxes)
    print(len(bounding_boxes))
    print("Remaining Bounding Box Coordinates:")
    for i, box in enumerate(bounding_boxes):
        print(f"Bounding Box {i+1}: {box}")
        cropped_part = crop_bounding_box(original_image, box)
        # cv2_imshow(cropped_part)
        cv2.imwrite(f"images/cropped_image_{i}.jpg", cropped_part)
    return bounding_boxes


# result = []
# print(len(bounding_boxes))
# for i, bbox in enumerate(pred_boxes):
#     x1, y1, x2, y2 = bbox



**cropping the images**


**OPENAI**

In [ ]:
import os
os.environ['OPENAI_API_KEY'] =  "<your key>"


In [ ]:
import instructor
from instructor import Mode
from openai import OpenAI
from pydantic import BaseModel, Field
from pprint import pprint
import base64
# from dotenv import load_dotenv

# load_dotenv()

client = instructor.patch(OpenAI(), mode=Mode.MD_JSON)
# reading the image and encoding it into encode it into base64
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

class ObjectDetection(BaseModel):
    # """
    # You are an object detection expert.
    # From the given list ,['High Rust', 'Low Rust', 'Medium Rust'] .
    # First identify if the object exist in the image. If object exist then provide the bounding box of the object and object name.
    # """
    x: int  = Field(description="x coordinate of detected object", default=0)
    y: int = Field(description="y coordinate of detected object", default=0)
    object_found_details: str = Field(description="Details of detected object.", default="")
    image_description: str = Field(descripion="Description of image.", default="")

#fn for calling gpt 4 for an image
def ask_gpt4_vision(system_instrutions, question, image_path):
    base64_image = encode_image(image_path)

    detected = client.chat.completions.create(
        response_model=ObjectDetection,
        model="gpt-4-turbo",
        max_tokens=200,
        messages=[
            {
                "role": "system",
                "content": system_instrutions
            },
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": question},
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{base64_image}"
                        },
                    },
                ],
            }
        ],
    )

    return {"x": detected.x, "y": detected.y, "object_found_details": detected.object_found_details, "image_description": detected.image_description}



# question = "Can you tell me type of rust on the image."
#coordinates = ask_gpt4_vision(system_instructions, question, image_path)
# to call the gpt 4 on the cropped images
#for call ing gpt4 on number of images present in the folder
def gpt4_vision_call(resized_path, system_instructions, questions):
  file_list = os.listdir(resized_path)
  file_list.sort()
  detected_labels = []
  # print(file_list)
# reading the file one by one in folder
  for files in file_list:
    # checking for the extension of the file
        if files.endswith(".jpg") or files.endswith(".png"):
          image_path = os.path.join(resized_path, files)
          print(image_path)
          coordinates = ask_gpt4_vision(system_instructions, questions, image_path)
          print(coordinates['object_found_details'], "-->", coordinates['image_description'])
          detected_labels.append(coordinates['object_found_details'])
  return detected_labels


# for calling the above two functions with prompt
def call_gpt(im, bounding_boxes = None):

    system_instructions1 =   """
    As an expert in object detection, you specialize in pinpointing specific elements within images.
    Given your expertise, you can efficiently analyze the provided image for any items listed, such as ['High Rust', 'Low Rust', 'Medium Rust', 'No Rust'].
    Rust is formed due to the corrosion on metal surface of electric pole, and it is of red-brownish color. It destroys the metal surface.
    Below is the deifinition of different types of rust:
    Once identified, you will accurately mark their presence by providing both the bounding box coordinates and the name of the detected object.
    You have to identify all the different areas of rust
    It is crucial to exclude any  information from the image to adhere to privacy and data protection standards.
    Output should only contain the type of rust no description is needed.
    """
    question = "Can you tell me type of rust on the image."

    labels = gpt4_vision_call(resized_path = '/content/images',system_instructions =  system_instructions1, questions = question)

  # if bounding boxes are present.
    if bounding_boxes !=None:
      bounding_boxes_array = np.array(bounding_boxes)
      annotated_frame_1 = annotate1(image_source=im, boxes=bounding_boxes_array,  phrases=labels)
      annotated_frame = annotated_frame_1[...,::-1]
      rgb_image = cv2.cvtColor(annotated_frame, cv2.COLOR_BGR2RGB)
      rgb_image1 = cv2.cvtColor(rgb_image, cv2.COLOR_BGR2RGB)
      cv2.imwrite('result.jpg', rgb_image1)

      return rgb_image1
    else:
      print("Image has no visible sign of rust")
      return im

# try:
#   os.mkdir('/content/output')
# except:
#   pass

# cv2.imwrite('/content/output/result.jpg', annotated_frame)


# import matplotlib.pyplot as plt
# plt.imshow(rgb_image)
# plt.axis('on')  # Hide axis
# plt.show()


In [ ]:
# reading the image and annotating it
im = cv2.imread('/content/drive/MyDrive/rust/Gemini_Generated_Image_7x4h7d7x4h7d7x4h.jpg')
#im = cv2.imread('/content/drive/MyDrive/rust/Gemini_Generated_Image_6mhtol6mhtol6mht.jpg')# no rust
im = cv2.resize(im, (1024, 1024))
bounding_boxes = saving_croppedImg(im)
if bounding_boxes == []:
  image_annotated = call_gpt(im)
else:
  image_annotated = call_gpt(im, bounding_boxes)
# image_annotated, bounding_boxes


[[90.058395, 160.0482, 633.0455, 1024.0], [98.75448, 151.80304, 379.91147, 850.0121], [301.26562, 219.20546, 623.15497, 947.07825], [91.48371, 495.94437, 170.08847, 988.90985], [350.6838, 188.31682, 639.68085, 674.59204], [71.406685, 183.63828, 347.80167, 558.8877], [137.68114, 187.50072, 614.6715, 601.78973], [292.08713, 559.0202, 620.3451, 993.68005]]
8
4
Remaining Bounding Box Coordinates:
Bounding Box 1: [98.75448, 151.80304, 379.91147, 850.0121]
Bounding Box 2: [91.48371, 495.94437, 170.08847, 988.90985]
Bounding Box 3: [350.6838, 188.31682, 639.68085, 674.59204]
Bounding Box 4: [292.08713, 559.0202, 620.3451, 993.68005]
/content/images/cropped_image_0.jpg
High Rust --> The image shows a metal surface with extensive corrosion displaying red-brownish rust, categorized as 'High Rust'.
/content/images/cropped_image_1.jpg
High Rust --> A vertical strip of metal with a significant presence of deep red-brownish rust concentrated in the middle, indicating a high level of corrosion.
/cont